In [15]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [25]:
import os, datetime
import keras
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from keras import layers

In [26]:
DATASET_ROOT = '../dataset'
TRAIN_DATA = os.path.join(DATASET_ROOT, 'train')
VAL_DATA = os.path.join(DATASET_ROOT, 'validation')
EVAL_DATA = os.path.join(DATASET_ROOT, 'test')

patience = 20
imageSizeX = 256
imageSizeY = 256
batchSize = 32

In [27]:
def datasets():
    train = ImageDataGenerator(
        rescale=1/255. ,
        rotation_range=40,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        vertical_flip=True,
    ).flow_from_directory(
        batch_size=batchSize,
        directory=TRAIN_DATA,
        shuffle=True,
        target_size=(imageSizeX, imageSizeY), 
        subset="training",
        class_mode='binary'
    )

    validation = ImageDataGenerator(
        rescale=1./255
    ).flow_from_directory(
        batch_size=batchSize,
        directory=VAL_DATA,
        shuffle=True,
        target_size=(imageSizeX, imageSizeY), 
        subset="training",
        class_mode='binary'
    )
    return train, validation


In [28]:
def create_model():
    model = keras.Sequential([

        layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(imageSizeX, imageSizeY, 3)),
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Dropout(0.2),

        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Dropout(0.2),

        layers.Conv2D(128, 3, padding='same', activation='relu'),
        layers.Conv2D(128, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Dropout(0.2),
        
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        
        layers.Dense(1, activation ='sigmoid')
    ])
    return model

In [29]:
class myCallback(keras.callbacks.Callback): 
    targetAccuracy = 0.9
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy') > (self.targetAccuracy)):
            self.model.stop_training = True

In [30]:
def train_model():

    train_data, val_data = datasets()
    model = create_model()

    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    model.fit(
        train_data,
        epochs=10,
        validation_data=val_data,
        callbacks=[tensorboard_callback]
    )
    model.save('cats_dogs_auto_multiConvLayer.h5')

In [33]:
%tensorboard --logdir logs
train_model()

Reusing TensorBoard on port 6006 (pid 271), started 0:00:56 ago. (Use '!kill 271' to kill it.)

Found 2999 images belonging to 2 classes.
Found 2048 images belonging to 2 classes.
Epoch 1/10


ResourceExhaustedError:  OOM when allocating tensor with shape[131072,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node RMSprop/RMSprop/update_12/mul_2 (defined at <ipython-input-30-0b82284c5ab3>:19) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_12394]

Function call stack:
train_function
